0. Sallitaan etäyhteydet
1. Etsi kirjasto jolla saa mysql:ään yhteyden pythonilla
2. Vedä data pandasin dataframeksi
3. Yhdistä eri taulujen dataframet yhdeksi
4. Kokeillaan visualisointia

In [74]:
# Tuodaan kirjastot
import pandas as pd
from pymysql import cursors

In [75]:
# Yhteysasetukset
settings = {
    "host": "172.31.17.68",
    "user": "root",
    "password": "Passw0rd",
    "db": "world"
}

# Otetaan yhteys
connection = pymysql.connect(**settings)

# Queryt
get_city = "SELECT * FROM city;"
get_country = "SELECT * FROM country;"
get_language = "SELECT * FROM countrylanguage;"

# Luodaan dataframet
cities_df = pd.read_sql(get_city, connection)
countries_df = pd.read_sql(get_country, connection)
languages_df = pd.read_sql(get_language, connection)

In [76]:
# Katsotaan, mitä sarakkeita on olemassa
cities_df.columns

Index(['ID', 'Name', 'CountryCode', 'District', 'Population'], dtype='object')

In [77]:
countries_df.columns

Index(['Code', 'Name', 'Continent', 'Region', 'SurfaceArea', 'IndepYear',
       'Population', 'LifeExpectancy', 'GNP', 'GNPOld', 'LocalName',
       'GovernmentForm', 'HeadOfState', 'Capital', 'Code2'],
      dtype='object')

In [78]:
cities_df.head()

,ID,Name,CountryCode,District,Population
0,1,Kabul,AFG,Kabol,1780000
1,2,Qandahar,AFG,Qandahar,237500
2,3,Herat,AFG,Herat,186800
3,4,Mazar-e-Sharif,AFG,Balkh,127800
4,5,Amsterdam,NLD,Noord-Holland,731200


In [43]:
# Poistetaan tarpeeton ID-sarake
del cities_df["ID"]
# korjaa
# Katsotaan mitä yhteisiä sarakkeita kahdella dataframella on
cols_for_city_country = cities_df.columns - countries_df.columns

C:\Users\mikkok\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: FutureWarning: using '-' to provide set differences with Indexes is deprecated, use .difference()
  from ipykernel import kernelapp as app


In [52]:
# Yhdistetään city-dataframen ainutlaatuiset sarakkeet country-dataframeen
city_country_df = pd.merge(cities_df[cols_for_city_country], countries_df, how = "inner", left_on = "CountryCode", right_on="Code")

In [56]:
# Yhdistetään city-maa-dataframe kieli-dataframeen
world_df = pd.merge(city_country_df, languages_df, on = "CountryCode")

# Poistetaan duplikaattisarake
del world_df["Code"]

In [65]:
world_df.head()

,CountryCode,District,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2,Language,IsOfficial,Percentage
0,AFG,Kabol,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF,Balochi,F,0.9
1,AFG,Kabol,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF,Dari,T,32.1
2,AFG,Kabol,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF,Pashto,T,52.4
3,AFG,Kabol,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF,Turkmenian,F,1.9
4,AFG,Kabol,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF,Uzbek,F,8.8


In [66]:
world_df.columns

Index(['CountryCode', 'District', 'Name', 'Continent', 'Region', 'SurfaceArea',
       'IndepYear', 'Population', 'LifeExpectancy', 'GNP', 'GNPOld',
       'LocalName', 'GovernmentForm', 'HeadOfState', 'Capital', 'Code2',
       'Language', 'IsOfficial', 'Percentage'],
      dtype='object')

In [69]:
from bokeh.charts import Histogram, BoxPlot, show
from bokeh.io import output_notebook, output_file
from bokeh.resources import INLINE

In [70]:
output_notebook()

Loading BokehJS ...

In [72]:
#rikki
p = BoxPlot(world_df, values = "GNP", title = "GNP")
show(p)